# 17기 KNN 정규세션 과제

## KNN 구현해보기
### 1. Preprocssing / EDA
지금까지 배운 내용을 토대로 해당 데이터에 대해 자유롭게 전처리와 EDA를 진행해주세요.
### 2. KNN 구현 & 파라미터 튜닝
수업 내용 및 실습 자료를 참고하여 KNN을 구현하고 파라미터 튜닝을 하며 결과를 비교해주세요.
### 3. Evaluation
결과에 대한 평가를 진행하고, 나름의 해석을 달아주세요.

**데이터:** [blackfriday | Kaggle](https://www.kaggle.com/llopesolivei/blackfriday)

---

## 0. 데이터 불러오기 및 전처리

In [7]:
import pandas as pd
df = pd.read_csv("blackfriday.csv", index_col = 0)
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1001088,P00046042,F,0-17,10,A,3,0,5,17.0,NaN,2010
1,1004493,P00347742,F,0-17,10,A,1,0,7,NaN,NaN,4483
2,1005302,P00048942,F,0-17,10,A,1,0,1,4.0,NaN,7696
3,1001348,P00145242,F,0-17,10,A,3,0,2,4.0,NaN,16429
4,1001348,P00106742,F,0-17,10,A,3,0,3,5.0,NaN,5780


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 0 to 4997
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   User_ID                     4998 non-null   int64  
 1   Product_ID                  4998 non-null   object 
 2   Gender                      4998 non-null   object 
 3   Age                         4998 non-null   object 
 4   Occupation                  4998 non-null   int64  
 5   City_Category               4998 non-null   object 
 6   Stay_In_Current_City_Years  4998 non-null   object 
 7   Marital_Status              4998 non-null   int64  
 8   Product_Category_1          4998 non-null   int64  
 9   Product_Category_2          3465 non-null   float64
 10  Product_Category_3          1544 non-null   float64
 11  Purchase                    4998 non-null   int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 507.6+ KB


In [9]:
df.drop(['User_ID',
         'Product_ID',
         'Product_Category_2','Product_Category_3'],axis=1, inplace=True)
#결측치가 너무 많은 'Product_Category_2','Product_Category_3' 는 제거
#필요없는 User_ID	Product_ID	 역시 제거

In [15]:
print(df['Age'].unique())
print(df['City_Category'].unique())
print(df['Stay_In_Current_City_Years'].unique())

['0-17' '18-25' '26-35' '36-45' '46-50' '51-55' '55+']
['A' 'B' 'C']
['3' '1' '2' '4+' '0']


In [19]:
df['Age']= df['Age'].replace(['0-17', '18-25' ,'26-35' ,'36-45' ,'46-50' ,'51-55' ,'55+'],
                            [0,1,2,3,4,5,6])
df['City_Category']= df['City_Category'].replace(['A' ,'B', 'C'],
                            [0,1,2])
df['Stay_In_Current_City_Years']= df['Stay_In_Current_City_Years'].replace(['3', '1', '2', '4+' ,'0'],
                            [3,1,2,4,0])
df['Gender']= df['Gender'].replace([ 'M' ,'F'],
                            [0,1])

object타입들이 전부 순서를 가진 범주형 변수이기에 그에 맞게 정수형으로 데이터 변경

## 데이터 분리

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:7], df.iloc[:,7], test_size=0.3, random_state=2021)



In [37]:
df.describe()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Purchase
count,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000
mean,0.246699,2.495798,8.036815,1.042817,1.856743,0.421369,5.320128,9234.655462
std,0.431133,1.353285,6.442697,0.760075,1.283462,0.493828,3.912281,4999.225081
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,13.000000
25%,0.000000,2.000000,2.000000,0.000000,1.000000,0.000000,1.000000,5497.750000
50%,0.000000,2.000000,7.000000,1.000000,2.000000,0.000000,5.000000,8049.000000
75%,0.000000,3.000000,14.000000,2.000000,3.000000,1.000000,8.000000,12039.000000
max,1.000000,6.000000,20.000000,2.000000,4.000000,1.000000,20.000000,23913.000000


데이터 분포를 보아 원래 숫자형이었던 occupitaion이 표준화를 할 필요 없다고 판단하여 안한다.

In [44]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=5)
neigh.fit(X_train, y_train)

KNeighborsRegressor()

In [45]:
neigh.score(X_test, y_test)

0.39822741047182597

In [48]:
for i in range(1,15):
    neigh = KNeighborsRegressor(n_neighbors=i)
    neigh.fit(X_train, y_train)
    print(i,neigh.score(X_test, y_test))

1 0.14047253356693923
2 0.3062659114751465
3 0.36875864311171525
4 0.39758774837648836
5 0.39822741047182597
6 0.4240442346496103
7 0.42368879457108843
8 0.42272858906841637
9 0.42496989953129816
10 0.4185853799287784
11 0.41788670799807515
12 0.4148043405391839
13 0.4095558662800408
14 0.4050131953139108


n_neighbors 가 9 인 것이 가장 성능이 높게 나타났다